In [1]:
import math
import time
import wandb
import spam
import pprint

In [2]:
#spam.mlp_classifier((784, 30, 10), epochs=1)

In [3]:
#spam.reduce_precision(1.3, 8)
dir(spam)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'error',
 'mlp_classifier',
 'reduce_precision',
 'system']

In [4]:
from distutils.sysconfig import get_python_lib
print(get_python_lib())

c:\users\lubko\appdata\local\programs\python\python39\Lib\site-packages


In [5]:
#import importlib
#importlib.reload(spam)

In [6]:
wandb.login()
#wandb.init(project="my-test-project")

wandb: Currently logged in as: lubomirkurcak (use `wandb login --relogin` to force relogin)


True

In [7]:
# vytvorenie sweep configu (pouzivame nahodne zvolenie parametrov)
sweep_config = {
    'method': 'random'
}

In [8]:
#metric = {
#    'name': 'loss',
#    'goal': 'minimize'
#}

metric = {
    'name': 'correct_classifications',
    'goal': 'maximize'
}

sweep_config['metric'] = metric

In [9]:
# diskretne parametre
parameters_dict = {
    'fp_subnormal': {
        'values': ['CPFLOAT_SUBN_RND',
                   'CPFLOAT_SUBN_USE']
    },
    
    'fp_round': {
        'values': ['CPFLOAT_RND_NA',
                   'CPFLOAT_RND_NZ',
                   'CPFLOAT_RND_NE',
                   'CPFLOAT_RND_TP',
                   'CPFLOAT_RND_TN',
                   'CPFLOAT_RND_TZ',
                   'CPFLOAT_RND_SP',
                   'CPFLOAT_RND_SE',
                   'CPFLOAT_RND_OD',
                   'CPFLOAT_NO_RND']
    },
    
    'fp_flip': {
        'values': ['CPFLOAT_NO_SOFTERR',
                   'CPFLOAT_SOFTERR']
    },
    
    'fp_explim': {
        'values': ['CPFLOAT_EXPRANGE_STOR',
                   'CPFLOAT_EXPRANGE_TARG']
    },
}

sweep_config['parameters'] = parameters_dict

In [10]:
# konstantne parametre
parameters_dict.update({
    'layer_sizes': {
        'value': (784, 30, 10)
    },
    'epochs': {
        'value': 5
    },
    'minibatch_size': {
        'value': 10
    },
    'learning_rate': {
        'value': 0.1
    },
    'weight_decay': {
        'value': 0.0001
    },
    'dropout': {
        'value': 0
    },
    'momentum_coefficient': {
        'value': 0
    },
    'loss': {
        'value': 'mean_squared_error'
    }
})

In [11]:
'''
precision
   * @brief Bits of precision of target format.
   *
   * @details The maximum values allowed are 24 and 53 if the storage format is
   * `float` or `double`, respectively.
'''

'''
maximum exponent
   * @brief Maximum exponent of target format.
   *
   * @details The maximum values allowed are 127 and 1023 if the storage format
   * is `float` or `double`, respectively. Larger values are reduced to the
   * maximum allowed value without warning. This field is ignored unless
   * `explim` is set to `CPFLOAT_EXPRANGE_TARG`.   
'''

# spojite parametre
parameters_dict.update({
    'fp_p': {
        'distribution': 'uniform',
        'min': 0,
        'max': 0.5
    },
    
    'fp_precision': {
        'distribution': 'q_uniform',
        'q': 1,
        'min': 4,
        'max': 24,
    },
    
    'fp_emax': {
        'distribution': 'q_uniform',
        'q': 1,
        'min': 4,
        'max': 127,
    },
})


'''
'fp_precision': {
    'distribution': 'q_log_uniform',
    'q': 1,
    'min': math.log(4),
    'max': math.log(24),
},

'fp_emax': {
    'distribution': 'q_log_uniform',
    'q': 1,
    'min': math.log(4),
    'max': math.log(127),
},
'''

"\n'fp_precision': {\n    'distribution': 'q_log_uniform',\n    'q': 1,\n    'min': math.log(4),\n    'max': math.log(24),\n},\n\n'fp_emax': {\n    'distribution': 'q_log_uniform',\n    'q': 1,\n    'min': math.log(4),\n    'max': math.log(127),\n},\n"

In [12]:
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'correct_classifications'},
 'parameters': {'dropout': {'value': 0},
                'epochs': {'value': 5},
                'fp_emax': {'distribution': 'q_uniform',
                            'max': 127,
                            'min': 4,
                            'q': 1},
                'fp_explim': {'values': ['CPFLOAT_EXPRANGE_STOR',
                                         'CPFLOAT_EXPRANGE_TARG']},
                'fp_flip': {'values': ['CPFLOAT_NO_SOFTERR',
                                       'CPFLOAT_SOFTERR']},
                'fp_p': {'distribution': 'uniform', 'max': 0.5, 'min': 0},
                'fp_precision': {'distribution': 'q_uniform',
                                 'max': 24,
                                 'min': 4,
                                 'q': 1},
                'fp_round': {'values': ['CPFLOAT_RND_NA',
                                        'CPFLOAT_RND_NZ',
                     

In [13]:
spam.system('echo %TIME%')

0

In [14]:
# inicializuj sweep pomocou konfigu (vytvori sa stranka sweepu, kde je mozne monitorovat behy a analyzovat parametre)
sweep_id = wandb.sweep(sweep_config, project="my-test-project")

Create sweep with ID: 9zf00zig
Sweep URL: https://wandb.ai/lubomirkurcak/my-test-project/sweeps/9zf00zig


In [15]:
# trenovacia funkcia, pouzita pri volani trenovacich agentov
def train():
    
    # inicialuzuj wandb a vyber config s nastavenymi hodnotami podla sweep configu
    run = wandb.init()
    config = run.config
    
    '''
    for dummy_epoch in range(30):
        wandb.log({"loss": 0.9 ** dummy_epoch,
                   "accuracy": 1 - 0.97 ** dummy_epoch}, commit=False, step=dummy_epoch)
    '''
    
    correct_classifications = spam.mlp_classifier(tuple(config["layer_sizes"]),
        epochs=config["epochs"],
        minibatch_size=config["minibatch_size"],
        learning_rate=config["learning_rate"],
        weight_decay=config["weight_decay"],
        dropout=config["dropout"],
        momentum_coefficient=config["momentum_coefficient"],
        fp_precision=config["fp_precision"],
        fp_emax=config["fp_emax"],
        fp_subnormal=config["fp_subnormal"],
        fp_round=config["fp_round"],
        fp_flip=config["fp_flip"],
        fp_p=config["fp_p"],
        fp_explim=config["fp_explim"]
    )
    
    
    wandb.log({'correct_classifications': correct_classifications}, commit=False, step=0)
    
    '''
    print('Early accuracy stop!')
    wandb.alert(
        title='Early accuracy stop!', 
        text=f'Accuracy {correct/total} is above 0.7',
        level=AlertLevel.INFO, # INFO, WARN, or ERROR 
        wait_duration=timedelta(minutes=1)
    )
    '''
        
    run.finish()
    

In [16]:
#spam.create_forward_net([784, 30, 10])
#spam.create_forward_net((784, 30, 10))

In [17]:
#wandb.agent("i91zir54", project="my-test-project", function=train, count=400)

In [18]:
wandb.agent(sweep_id, train, count=4)

wandb: Agent Starting Run: rawcnorw with config:
wandb: 	dropout: 0
wandb: 	epochs: 5
wandb: 	fp_emax: 13
wandb: 	fp_explim: CPFLOAT_EXPRANGE_STOR
wandb: 	fp_flip: CPFLOAT_NO_SOFTERR
wandb: 	fp_p: 0.17930676012392943
wandb: 	fp_precision: 20
wandb: 	fp_round: CPFLOAT_RND_TZ
wandb: 	fp_subnormal: CPFLOAT_SUBN_USE
wandb: 	layer_sizes: [784, 30, 10]
wandb: 	learning_rate: 0.1
wandb: 	loss: mean_squared_error
wandb: 	minibatch_size: 10
wandb: 	momentum_coefficient: 0
wandb: 	weight_decay: 0.0001
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


cpfloat_validate_optstruct returned 0
Epoch 0/5 Batch 0/6000 Cost 2.962585 LR 0.100000
Epoch 0/5 Batch 1024/6000 Cost 0.440806 LR 0.100000
Epoch 0/5 Batch 2048/6000 Cost 0.144906 LR 0.100000
Epoch 0/5 Batch 3072/6000 Cost 0.339243 LR 0.100000
Epoch 0/5 Batch 4096/6000 Cost 0.311939 LR 0.100000
Epoch 0/5 Batch 5120/6000 Cost 0.230707 LR 0.100000
Epoch 0/5 Correct 8911/10000 Best 8911(0)Epoch 1/5 Batch 0/6000 Cost 0.091082 LR 0.100000
Epoch 1/5 Batch 1024/6000 Cost 0.410295 LR 0.100000
Epoch 1/5 Batch 2048/6000 Cost 0.089125 LR 0.100000
Epoch 1/5 Batch 3072/6000 Cost 0.320241 LR 0.100000
Epoch 1/5 Batch 4096/6000 Cost 0.239251 LR 0.100000
Epoch 1/5 Batch 5120/6000 Cost 0.235762 LR 0.100000
Epoch 1/5 Correct 9035/10000 Best 9035(0)Epoch 2/5 Batch 0/6000 Cost 0.080951 LR 0.100000
Epoch 2/5 Batch 1024/6000 Cost 0.423651 LR 0.100000
Epoch 2/5 Batch 2048/6000 Cost 0.076016 LR 0.100000
Epoch 2/5 Batch 3072/6000 Cost 0.280879 LR 0.100000
Epoch 2/5 Batch 4096/6000 Cost 0.215585 LR 0.100000
Epoch

wandb: Network error (SSLError), entering retry loop. See W:\rpnn\python\wandb\run-20220228_230338-rawcnorw\logs\debug-internal.log for full traceback.


Epoch 4/5 Correct 9199/10000 Best 9199(0)

correct_classifications,9199
_runtime,244
_timestamp,1646086062
_step,0


...

cpfloat_validate_optstruct returned 0
Epoch 0/5 Batch 0/6000 Cost 3.573412 LR 0.100000
Epoch 0/5 Batch 1024/6000 Cost 0.474264 LR 0.100000
Epoch 0/5 Batch 2048/6000 Cost 0.131969 LR 0.100000
Epoch 0/5 Batch 3072/6000 Cost 0.399202 LR 0.100000
Epoch 0/5 Batch 4096/6000 Cost 0.418867 LR 0.100000
Epoch 0/5 Batch 5120/6000 Cost 0.345642 LR 0.100000
Epoch 0/5 Correct 8237/10000 Best 8237(0)Epoch 1/5 Batch 0/6000 Cost 0.190224 LR 0.100000
Epoch 1/5 Batch 1024/6000 Cost 0.411175 LR 0.100000
Epoch 1/5 Batch 2048/6000 Cost 0.095686 LR 0.100000
Epoch 1/5 Batch 3072/6000 Cost 0.332063 LR 0.100000
Epoch 1/5 Batch 4096/6000 Cost 0.311516 LR 0.100000
Epoch 1/5 Batch 5120/6000 Cost 0.284581 LR 0.100000
Epoch 1/5 Correct 8798/10000 Best 8798(0)Epoch 2/5 Batch 0/6000 Cost 0.116325 LR 0.100000
Epoch 2/5 Batch 1024/6000 Cost 0.383890 LR 0.100000
Epoch 2/5 Batch 2048/6000 Cost 0.082288 LR 0.100000
Epoch 2/5 Batch 3072/6000 Cost 0.318543 LR 0.100000
Epoch 2/5 Batch 4096/6000 Cost 0.267060 LR 0.100000
Epoch

wandb: Network error (SSLError), entering retry loop. See W:\rpnn\python\wandb\run-20220228_233055-o0spfflw\logs\debug-internal.log for full traceback.


Epoch 4/5 Batch 1024/6000 Cost 0.315740 LR 0.100000
Epoch 4/5 Batch 2048/6000 Cost 0.079891 LR 0.100000
Epoch 4/5 Batch 3072/6000 Cost 0.306751 LR 0.100000
Epoch 4/5 Batch 4096/6000 Cost 0.251749 LR 0.100000
Epoch 4/5 Batch 5120/6000 Cost 0.179256 LR 0.100000
Epoch 4/5 Correct 9107/10000 Best 9107(0)

correct_classifications,9107
_runtime,647
_timestamp,1646088102
_step,0


correct_classifications,▁
_runtime,▁
_timestamp,▁
_step,▁


In [19]:
'''
import gym
env = gym.make('CartPole-v0')

for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        #print(observation)
        #action = env.action_space.sample()
        #action = (t+1)//2%2
        action = 1 if observation[2] > 0 else 0

        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()
'''

'\nimport gym\nenv = gym.make(\'CartPole-v0\')\n\nfor i_episode in range(20):\n    observation = env.reset()\n    for t in range(100):\n        env.render()\n        #print(observation)\n        #action = env.action_space.sample()\n        #action = (t+1)//2%2\n        action = 1 if observation[2] > 0 else 0\n\n        observation, reward, done, info = env.step(action)\n        if done:\n            print("Episode finished after {} timesteps".format(t+1))\n            break\nenv.close()\n'

wandb: Network error resolved after 0:10:33.677092, resuming normal operation.
